In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WY_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2724797100,WY,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,9692597008,WY,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,6083237006,WY,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,7125427008,WY,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,8862647107,WY,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2724797100,WY,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,9692597008,WY,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,6083237006,WY,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,7125427008,WY,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,8862647107,WY,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
721,6427467010,WY,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1377,5725807009,WY,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1447,9082927003,WY,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1728,6664317004,WY,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1903,5288408603,WY,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('56')] #WY
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/5 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

5


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14909/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
721,6427467010,WY,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,[]
1377,5725807009,WY,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,[]
1447,9082927003,WY,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,[]
1728,6664317004,WY,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,[]
1903,5288408603,WY,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14909/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
721,6427467010,WY,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1377,5725807009,WY,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1447,9082927003,WY,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1728,6664317004,WY,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1903,5288408603,WY,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
721,6427467010,WY,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1377,5725807009,WY,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1447,9082927003,WY,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1728,6664317004,WY,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1903,5288408603,WY,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
721,6427467010,NaN
1377,5725807009,NaN
1447,9082927003,NaN
1728,6664317004,NaN
1903,5288408603,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WY_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,963407,2724797100,WY,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,963408,9692597008,WY,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,963409,6083237006,WY,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,963410,7125427008,WY,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,963411,8862647107,WY,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,963407,2724797100,WY,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14,NaN
1,963408,9692597008,WY,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14,NaN
2,963409,6083237006,WY,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14,NaN
3,963410,7125427008,WY,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14,NaN
4,963411,8862647107,WY,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14,NaN
...,...,...,...,...,...,...,...,...
2132,965539,4395967002,WY,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,5.602500e+14,NaN
2133,965540,6985647108,WY,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,5.603797e+14,NaN
2134,965541,7996438405,WY,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,5.600500e+14,NaN
2135,965542,9054647103,WY,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,5.603300e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,963407,2724797100,WY,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14,NaN,5.600500e+14
1,963408,9692597008,WY,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14,NaN,5.600196e+14
2,963409,6083237006,WY,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14,NaN,5.600500e+14
3,963410,7125427008,WY,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14,NaN,5.604198e+14
4,963411,8862647107,WY,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14,NaN,5.600996e+14


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2724797100,WY,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,9692597008,WY,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,6083237006,WY,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,7125427008,WY,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,8862647107,WY,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14
...,...,...,...,...,...,...
2132,4395967002,WY,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,5.602500e+14
2133,6985647108,WY,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,5.603797e+14
2134,7996438405,WY,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,5.600500e+14
2135,9054647103,WY,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,5.603300e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
721,6427467010,WY,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1377,5725807009,WY,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1447,9082927003,WY,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1728,6664317004,WY,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1903,5288408603,WY,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN


Convert to File

In [19]:
fips_WY = fips_merge_drop

In [20]:
fips_WY.to_csv("../../../data/state_data/geo/geo_fips/23/WY_fips_scraped.csv") 